In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from textblob import TextBlob
import textblob as tb
import json
import nltk

In [15]:
pDFt = pd.read_csv("audio_opensummary2.csv", encoding='latin1')
pDFt.head()

,ID,Content
0,9890,my friend is a very good one and hes always th...
1,9766,a good friend is somebody who listens to you w...
2,9525,a good friend who is somebody that is always t...
3,9170,"a good friend is someone who is there for you,..."
4,9137,"oh I have a good friend umm, she's nice and sw..."


In [16]:
polarity = []
subjectivity = []
tags = []
for i in range(0, len(pDFt.ID)):
    T = TextBlob(str(pDFt.Content[i]))
    
    polarity.append(T.sentiment[0])
    subjectivity.append(T.sentiment[1])
    
    tags2=[]
    for word, tag in T.tags:
        tags2.append(tag)
    tags.append(tags2)
pDFt["POStags"] = tags
pDFt["Polarity"] = polarity
pDFt["Subjectivity"] = subjectivity

In [17]:
from empath import Empath
import re

#create list of all empath categories

lexicon = Empath()
emp = lexicon.analyze("Testing", normalize=True)
wordlist = []
for word, value in emp.items():
    wordlist.append(word)
print(wordlist)



['help', 'office', 'dance', 'money', 'wedding', 'domestic_work', 'sleep', 'medical_emergency', 'cold', 'hate', 'cheerfulness', 'aggression', 'occupation', 'envy', 'anticipation', 'family', 'vacation', 'crime', 'attractive', 'masculine', 'prison', 'health', 'pride', 'dispute', 'nervousness', 'government', 'weakness', 'horror', 'swearing_terms', 'leisure', 'suffering', 'royalty', 'wealthy', 'tourism', 'furniture', 'school', 'magic', 'beach', 'journalism', 'morning', 'banking', 'social_media', 'exercise', 'night', 'kill', 'blue_collar_job', 'art', 'ridicule', 'play', 'computer', 'college', 'optimism', 'stealing', 'real_estate', 'home', 'divine', 'sexual', 'fear', 'irritability', 'superhero', 'business', 'driving', 'pet', 'childish', 'cooking', 'exasperation', 'religion', 'hipster', 'internet', 'surprise', 'reading', 'worship', 'leader', 'independence', 'movement', 'body', 'noise', 'eating', 'medieval', 'zest', 'confusion', 'water', 'sports', 'death', 'healing', 'legend', 'heroic', 'celebr

In [18]:
#Empath features

for word in wordlist:
    pctt = []
    for i in range(0, pDFt.shape[0]):
        content = re.sub(r'[^\w\s]', '', str(pDFt.Content[i]).lower())
        lexicon = Empath()
        emp = lexicon.analyze(content, categories=[word], normalize = True)
        if emp != None:
            for key, value in emp.items():
                pctt.append(value)
        else:
            pctt.append(0)
    pDFt[word] = pctt

In [19]:
#get set of POS tags
posTags = []
for i in range(0, pDFt.shape[0]):
    for tag in pDFt.POStags[i]:
        posTags.append(tag)
posSet = set(posTags)
print(posSet)

{'WP', 'DT', 'NNP', 'POS', 'VBD', 'VBP', 'UH', 'CD', 'JJ', 'WDT', 'JJR', 'NN', 'NNS', 'VB', 'TO', 'WRB', 'CC', 'PDT', 'VBZ', 'PRP', 'RBS', 'VBG', 'RP', 'MD', 'VBN', 'JJS', 'RB', 'PRP$', 'EX', 'IN'}


In [20]:
#POS tag counting

poswordst = []
for posList in pDFt.POStags:
    poswordst.append(len(posList))
pDFt["WordsTags"] = poswordst

for tag in posSet:
    cnt = []
    for posList in pDFt.POStags:
        counter = 0
        for item in posList:
            if item == tag:
                counter += 1
        cnt.append(counter/len(posList))
    pDFt[tag] = cnt

In [21]:
#Volume features for Tweets
import re

wsum = []
csum = []

for i in range(0, pDFt.shape[0]):
    wsum.append(len(pDFt.Content[i].split(" ")))
    csum.append(len(pDFt.Content[i]))

pDFt["WordSum"] = wsum
pDFt["CharacterSum"] = csum

In [22]:
pDFt

,ID,Content,POStags,Polarity,Subjectivity,help,office,dance,money,wedding,...,RP,MD,VBN,JJS,RB,PRP$,EX,IN,WordSum,CharacterSum
0,9890,my friend is a very good one and hes always th...,"[PRP$, NN, VBZ, DT, RB, JJ, CD, CC, VBZ, RB, J...",0.542000,0.516000,0.090909,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.159091,0.022727,0.000000,0.022727,44,204
1,9766,a good friend is somebody who listens to you w...,"[DT, JJ, NN, VBZ, NN, WP, VBZ, TO, PRP, WRB, P...",0.231019,0.651389,0.035088,0.000000,0.000000,0.000000,0.017544,...,0.000000,0.017544,0.000000,0.000000,0.087719,0.035088,0.000000,0.017544,57,300
2,9525,a good friend who is somebody that is always t...,"[DT, JJ, NN, WP, VBZ, NN, WDT, VBZ, RB, EX, IN...",0.272222,0.444444,0.076923,0.000000,0.000000,0.000000,0.000000,...,0.111111,0.037037,0.000000,0.000000,0.074074,0.000000,0.037037,0.037037,27,129
3,9170,"a good friend is someone who is there for you,...","[DT, JJ, NN, VBZ, NN, WP, VBZ, RB, IN, PRP, WP...",0.183333,0.583333,0.022222,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.044444,0.000000,0.000000,0.022222,0.000000,0.000000,0.088889,45,212
4,9137,"oh I have a good friend umm, she's nice and sw...","[NN, PRP, VBP, DT, JJ, NN, NN, PRP, VBZ, JJ, C...",0.470000,0.710000,0.027778,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.025000,0.000000,0.025000,0.025000,0.000000,0.050000,36,168
5,8917,"describing a good friend, a good friend is a f...","[VBG, DT, JJ, NN, DT, JJ, NN, VBZ, DT, NN, WP,...",0.400000,0.550000,0.073171,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.048780,0.048780,0.000000,0.048780,41,210
6,8889,a good friend is just somebody who you would g...,"[DT, JJ, NN, VBZ, RB, NN, WP, PRP, MD, VB, IN,...",0.147727,0.420455,0.035398,0.000000,0.000000,0.000000,0.000000,...,0.008475,0.076271,0.016949,0.000000,0.101695,0.016949,0.000000,0.127119,113,581
7,8801,desribe a good friend. He's he's one of my bes...,"[VB, DT, JJ, NN, PRP, VBZ, PRP, VBZ, CD, IN, P...",0.800000,0.500000,0.071429,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.058824,0.058824,0.058824,0.000000,0.058824,14,72
8,8760,someone im thinking of who is a good friend of...,"[NN, NN, NN, IN, WP, VBZ, DT, JJ, NN, IN, NN, ...",0.257576,0.543939,0.028169,0.000000,0.014085,0.000000,0.014085,...,0.000000,0.000000,0.000000,0.013333,0.040000,0.013333,0.000000,0.160000,71,443
9,8759,"describe a good friend, ok, a good friend is a...","[VB, DT, JJ, NN, UH, DT, JJ, NN, VBZ, DT, JJ, ...",0.570000,0.660000,0.100000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.064516,0.000000,0.000000,0.032258,30,158


In [23]:
pDFt.to_csv("textfeatures2021.csv")